<div style="color:rgb(0,0,255);font-size: 40px;font-weight:700;">
MAIN SETTINGS
</div>

In [37]:
###SETTINGS###

import os


In [38]:
import sys

In [39]:
import re


In [40]:
import time


In [41]:
import subprocess


In [42]:
import shutil


In [43]:
from concurrent.futures import ThreadPoolExecutor, as_completed


In [44]:
from getpass import getpass


In [45]:
from urllib.parse import urlencode

# Platform detection


In [46]:
ON_VAST = any(k in os.environ for k in ("VAST_CONTAINERLABEL", "VAST_TCP_PORT_22", "CONTAINER_ID")) or os.path.exists('/workspace')




In [47]:
MAX_PARALLEL_DOWNLOADS = max(1, int(os.environ.get("MAX_PARALLEL_DOWNLOADS", "3")))


In [48]:
MIN_VALID_FILE_BYTES = int(os.environ.get("MIN_VALID_FILE_BYTES", "1000000"))



In [49]:
system_packages = [
    "aria2",
    "libjpeg-dev",
    "zlib1g-dev",
    "libpng-dev",
    "libfreetype6-dev",
    "libopenjp2-7-dev",
    "build-essential",
]

print("Checking/installing download and Pillow build dependencies...")
try:
    subprocess.run(["apt", "update", "-qq"], check=True, capture_output=True)
    result = subprocess.run(["apt", "install", "-y", "-qq", *system_packages], capture_output=True, text=True)
    if result.returncode == 0:
        print("System dependencies are ready")
    else:
        print("Install failed (code {}):".format(result.returncode))
        print("stderr:", result.stderr.strip())
except subprocess.CalledProcessError as e:
    print(f"apt error (code {e.returncode}): {e.stderr}")
except Exception as e:
    print(f"Unexpected error: {e}")

if shutil.which("aria2c") is None:
    raise RuntimeError("aria2c is still unavailable after apt install")

# Determining the working directory


aria2c already available


In [50]:
possible_bases = [
    "/workspace",
]



In [51]:
BASE_DIR = None


In [52]:
for path in possible_bases:
    if os.path.isdir(path):
        BASE_DIR = path
        break



In [53]:
if BASE_DIR is None:
    BASE_DIR = os.getcwd()
    print("WARNING: Known directory not found:", BASE_DIR)



In [54]:
print("Working directory:", BASE_DIR)

# Configuration


Working directory: /workspace


In [55]:
FORGE_DIR        = os.path.join(BASE_DIR, "stable-diffusion-webui-forge")


In [56]:
MODELS_DIR       = os.path.join(BASE_DIR, "stable-diffusion-webui-forge", "models", "Stable-diffusion")


In [57]:
LORA_DIR         = os.path.join(BASE_DIR, "stable-diffusion-webui-forge", "models", "Lora")


In [58]:
CONTROLNET_DIR   = os.path.join(FORGE_DIR, "extensions", "sd-webui-controlnet")


In [59]:
CONTROLNET_MODELS_DIR = os.path.join(CONTROLNET_DIR, "models")


In [60]:
EXTENSIONS_DIR   = os.path.join(FORGE_DIR, "extensions")


In [61]:
OUTPUTS_DIR      = os.path.join(FORGE_DIR, "outputs")


In [62]:
VOLUME_DIR       = os.path.join(BASE_DIR, "volume")


In [63]:
GEN_DIR          = os.path.join(BASE_DIR, "gen")


In [64]:
IMAGES_DIR       = os.path.join(GEN_DIR, "Images")



In [65]:
for d in [MODELS_DIR, LORA_DIR, CONTROLNET_DIR, CONTROLNET_MODELS_DIR, EXTENSIONS_DIR, OUTPUTS_DIR, VOLUME_DIR, GEN_DIR, IMAGES_DIR]:
    os.makedirs(d, exist_ok=True)

# Dependencies used by generation cell


In [66]:
for pkg in ["openpyxl", "requests"]:
    try:
        __import__(pkg)
    except ImportError:
        print(f"Installing missing dependency: {pkg}")
        subprocess.run(["python3", "-m", "pip", "install", "-q", pkg], check=True)

In [67]:
def get_secret(name: str):
    """Get secret from environment variables only."""
    value = os.environ.get(name)
    if value:
        return value.strip(), "env"
    return None, None




In [68]:
CIVITAI_TOKEN, CIVITAI_SRC = get_secret("CIVITAI_TOKEN")


In [69]:
HF_TOKEN, HF_SRC = get_secret("HF_TOKEN")



In [70]:
if not CIVITAI_TOKEN:
    manual_civitai = getpass("Enter CIVITAI_TOKEN (leave blank to skip): ").strip()
    if manual_civitai:
        CIVITAI_TOKEN, CIVITAI_SRC = manual_civitai, "manual_input"



In [71]:
if not HF_TOKEN:
    manual_hf = getpass("Enter HF_TOKEN (leave blank to skip): ").strip()
    if manual_hf:
        HF_TOKEN, HF_SRC = manual_hf, "manual_input"



In [72]:
TOKENS = {}


In [73]:
if CIVITAI_TOKEN:
    TOKENS["CIVITAI"] = CIVITAI_TOKEN


In [74]:
if HF_TOKEN:
    TOKENS["HF_TOKEN"] = HF_TOKEN



In [75]:
print("Token sources:")


Token sources:


In [76]:
print(f"  CIVITAI_TOKEN: {CIVITAI_SRC or 'not found'}")


  CIVITAI_TOKEN: env


In [77]:
print(f"  HF_TOKEN: {HF_SRC or 'not found'}")


  HF_TOKEN: env


In [78]:
if ON_VAST:
    print("Vast.ai tip: add CIVITAI_TOKEN/HF_TOKEN in template env vars, restart container, then rerun this cell.")



Vast.ai tip: add CIVITAI_TOKEN/HF_TOKEN in template env vars, restart container, then rerun this cell.


In [79]:
if not CIVITAI_TOKEN:
    print("CivitAI token not found")


In [80]:
if not HF_TOKEN:
    print("HF token not found")


In [81]:
if not TOKENS:
    raise RuntimeError("No tokens were provided. Set secrets or enter at least one token (CivitAI or HF).")




In [82]:
def _prepare_download_url(url, token):
    """CivitAI download works more reliably with token as query param."""
    if token and "civitai.com/api/download/models" in url and "token=" not in url:
        sep = "&" if "?" in url else "?"
        return f"{url}{sep}{urlencode({'token': token})}"
    return url




In [83]:
def _looks_valid_file(path, min_bytes=MIN_VALID_FILE_BYTES):
    return os.path.exists(path) and os.path.getsize(path) > min_bytes




In [84]:
def _human_mb(num_bytes):
    return f"{num_bytes / (1024 * 1024):.1f} MB"




In [85]:
def _estimate_expected_mb(label):
    # Examples: "151 MB", "6,46 GB"
    match = re.search(r"(\d+[\.,]?\d*)\s*(MB|GB)", label, re.IGNORECASE)
    if not match:
        return None
    value = float(match.group(1).replace(',', '.'))
    unit = match.group(2).upper()
    return value * (1024 if unit == "GB" else 1)




In [86]:
def _size_sanity_warning(path, expected_mb, tolerance=0.7):
    if expected_mb is None or not os.path.exists(path):
        return
    actual_mb = os.path.getsize(path) / (1024 * 1024)
    if actual_mb < expected_mb * tolerance:
        print(f"  WARNING: file size looks low ({actual_mb:.1f} MB vs expected ~{expected_mb:.1f} MB)")




In [87]:
def _has_min_free_disk(path, required_mb, reserve_mb=1024):
    if required_mb is None:
        return True
    usage = shutil.disk_usage(path)
    free_mb = usage.free / (1024 * 1024)
    return free_mb >= (required_mb + reserve_mb)




In [88]:
def _download_one(job, target_dir):
    label, url, filename, token_name = job
    token = TOKENS.get(token_name)
    output_path = os.path.join(target_dir, filename)

    expected_mb = _estimate_expected_mb(label)

    if _looks_valid_file(output_path):
        size = os.path.getsize(output_path)
        print(f"[SKIP] {label}: already exists ({_human_mb(size)})")
        _size_sanity_warning(output_path, expected_mb)
        return (label, True, "exists")

    if expected_mb is not None and not _has_min_free_disk(target_dir, expected_mb):
        return (label, False, "insufficient_disk")

    tmp_path = output_path + ".part"
    if os.path.exists(tmp_path):
        os.remove(tmp_path)

    final_url = _prepare_download_url(url, token if token_name == "CIVITAI" else None)

    # CivitAI signed links on Vast can return 403 with aggressive multi-connection mode.
    max_conn = "1" if token_name == "CIVITAI" else "16"
    split = "1" if token_name == "CIVITAI" else "16"

    cmd = [
        "aria2c",
        "--allow-overwrite=true",
        "--auto-file-renaming=false",
        "--continue=true",
        f"--max-connection-per-server={max_conn}",
        f"--split={split}",
        "--min-split-size=1M",
        "--console-log-level=warn",
        "--summary-interval=1",
        "--check-certificate=false",
        "--header=User-Agent: Mozilla/5.0",
        "--header=Referer: https://civitai.com/",
        "--out", os.path.basename(tmp_path),
        "--dir", target_dir,
        final_url,
    ]

    if token_name == "HF_TOKEN" and token:
        cmd.insert(-1, f"--header=Authorization: Bearer {token}")

    print(f"[DOWNLOADING] {label}")
    result = subprocess.run(cmd, text=True, capture_output=True)
    if result.returncode != 0:
        stderr = (result.stderr or "").strip()
        stdout = (result.stdout or "").strip()
        msg = stderr or stdout or f"aria2c exited {result.returncode}"

        # Retry once for CivitAI with curl fallback if aria2 still gets 403.
        if token_name == "CIVITAI":
            curl_cmd = [
                "curl", "-L", "--fail", "--retry", "2", "--retry-delay", "3",
                "-A", "Mozilla/5.0", "-e", "https://civitai.com/",
                "-o", tmp_path, final_url
            ]
            curl_res = subprocess.run(curl_cmd, text=True, capture_output=True)
            if curl_res.returncode == 0 and _looks_valid_file(tmp_path):
                os.replace(tmp_path, output_path)
                _size_sanity_warning(output_path, expected_mb)
                return (label, True, _human_mb(os.path.getsize(output_path)))
            curl_err = (curl_res.stderr or curl_res.stdout or "curl failed").strip()
            msg = f"{msg}\nFallback curl: {curl_err}"

        if os.path.exists(tmp_path):
            os.remove(tmp_path)
        return (label, False, msg)

    if not _looks_valid_file(tmp_path):
        size = os.path.getsize(tmp_path) if os.path.exists(tmp_path) else 0
        if os.path.exists(tmp_path):
            os.remove(tmp_path)
        return (label, False, f"downloaded file too small ({_human_mb(size)})")

    os.replace(tmp_path, output_path)
    _size_sanity_warning(output_path, expected_mb)
    return (label, True, _human_mb(os.path.getsize(output_path)))


In [89]:
def run_download_list(download_list, target_dir, title):
    print(f"\n=== {title} ===")
    os.makedirs(target_dir, exist_ok=True)

    if not download_list:
        print("No items.")
        return

    workers = min(MAX_PARALLEL_DOWNLOADS, len(download_list))
    print(f"Parallel downloads: {workers}")

    ok = 0
    fail = 0

    with ThreadPoolExecutor(max_workers=workers) as ex:
        futures = [ex.submit(_download_one, job, target_dir) for job in download_list]
        for fut in as_completed(futures):
            label, success, info = fut.result()
            if success:
                ok += 1
                print(f"[OK]   {label} -> {info}")
            else:
                fail += 1
                print(f"[FAIL] {label} -> {info}")

    print(f"Done: OK={ok}, FAIL={fail}")
    if fail > 0:
        raise RuntimeError(f"Some downloads failed in {title}: {fail} item(s)")




In [90]:
### OPTIONAL: FORGE BOOTSTRAP ###

import os


In [91]:
import shutil


In [92]:
import subprocess



In [93]:
required_packages = ["git", "python3-venv", "python3-pip", "libjpeg-dev", "zlib1g-dev", "libpng-dev", "build-essential"]


In [94]:
print("Checking/installing platform dependencies...")


Checking/installing platform dependencies...


In [95]:
subprocess.run(["apt", "update", "-qq"], check=False)


81 packages can be upgraded. Run 'apt list --upgradable' to see them.


CompletedProcess(args=['apt', 'update', '-qq'], returncode=0)

In [96]:
subprocess.run(["apt", "install", "-y", "-qq", *required_packages], check=False)



git is already the newest version (1:2.43.0-1ubuntu7.3).
python3-venv is already the newest version (3.12.3-0ubuntu2.1).
python3-venv set to manually installed.
python3-pip is already the newest version (24.0+dfsg-1ubuntu1.3).
0 upgraded, 0 newly installed, 0 to remove and 81 not upgraded.


CompletedProcess(args=['apt', 'install', '-y', '-qq', 'git', 'python3-venv', 'python3-pip'], returncode=0)

In [97]:
launch_script = os.path.join(FORGE_DIR, "webui.sh")


In [98]:
git_head = os.path.join(FORGE_DIR, ".git", "HEAD")


In [99]:
forge_ready = os.path.isfile(launch_script) and os.path.isfile(git_head)



In [100]:
if forge_ready:
    print("WebUI Forge already exists and looks valid, skipping clone.")
else:
    if os.path.isdir(FORGE_DIR):
        print("FORGE_DIR exists but WebUI Forge is incomplete/corrupted. Recreating...")
        shutil.rmtree(FORGE_DIR)

    print("Cloning WebUI Forge...")
    subprocess.run([
        "git", "clone", "https://github.com/lllyasviel/stable-diffusion-webui-forge", FORGE_DIR
    ], check=True)

    if not os.path.isfile(os.path.join(FORGE_DIR, "webui.sh")):
        raise FileNotFoundError("Clone completed but webui.sh not found. Check repository state.")



WebUI Forge already exists and looks valid, skipping clone.


In [101]:
print("Optional bootstrap finished.")


Optional bootstrap finished.


In [102]:
### CONTROLNET INSTALL OPTIONS ###

# Option A (default): force clean reinstall
# Option B: set CONTROLNET_INSTALL_MODE = "update" for git pull in existing repo
# Option C: set CONTROLNET_INSTALL_MODE = "skip" to keep current state

CONTROLNET_REPO_URL = "https://github.com/Mikubill/sd-webui-controlnet"


In [103]:
CONTROLNET_INSTALL_MODE = os.environ.get("CONTROLNET_INSTALL_MODE", "reinstall").strip().lower()



In [104]:
if CONTROLNET_INSTALL_MODE not in {"reinstall", "update", "skip"}:
    raise ValueError("CONTROLNET_INSTALL_MODE must be one of: reinstall, update, skip")



In [105]:
print(f"ControlNet extension path: {CONTROLNET_DIR}")


ControlNet extension path: /workspace/stable-diffusion-webui-forge/extensions/sd-webui-controlnet


In [106]:
print(f"Install mode: {CONTROLNET_INSTALL_MODE}")



Install mode: reinstall


In [107]:
if CONTROLNET_INSTALL_MODE == "skip":
    print("ControlNet install skipped")
else:
    if os.path.isdir(CONTROLNET_DIR) and CONTROLNET_INSTALL_MODE == "reinstall":
        print("Removing existing ControlNet directory...")
        shutil.rmtree(CONTROLNET_DIR)

    if not os.path.isdir(CONTROLNET_DIR):
        print("Cloning ControlNet repository...")
        subprocess.run(["git", "clone", CONTROLNET_REPO_URL, CONTROLNET_DIR], check=True)
    else:
        print("Updating ControlNet repository...")
        subprocess.run(["git", "-C", CONTROLNET_DIR, "pull", "--ff-only"], check=True)



Removing existing ControlNet directory...
Cloning ControlNet repository...


Cloning into '/workspace/stable-diffusion-webui-forge/extensions/sd-webui-controlnet'...


In [108]:
os.makedirs(CONTROLNET_MODELS_DIR, exist_ok=True)


In [109]:
print("ControlNet repository is ready")


ControlNet repository is ready


In [110]:
print(f"ControlNet models directory: {CONTROLNET_MODELS_DIR}")

#ControlNET models download



ControlNet models directory: /workspace/stable-diffusion-webui-forge/extensions/sd-webui-controlnet/models


In [111]:
controlnet_models_to_download = [
    ("t2i-adapter_xl_openpose 151 MB", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_xl_openpose.safetensors", "t2i-adapter_xl_openpose.safetensors", "HF_TOKEN"),
    ("t2i-adapter_xl_canny 148 MB", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_xl_canny.safetensors", "t2i-adapter_xl_canny.safetensors", "HF_TOKEN"),
    ("t2i-adapter_xl_sketch 148 MB", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_xl_sketch.safetensors", "t2i-adapter_xl_sketch.safetensors", "HF_TOKEN"),
    ("t2i-adapter_diffusers_xl_depth_midas 151 MB", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_diffusers_xl_depth_midas.safetensors", "t2i-adapter_diffusers_xl_depth_midas.safetensors", "HF_TOKEN"),
    ("t2i-adapter_diffusers_xl_depth_zoe 151 MB", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_diffusers_xl_depth_zoe.safetensors", "t2i-adapter_diffusers_xl_depth_zoe.safetensors", "HF_TOKEN"),
    ("t2i-adapter_diffusers_xl_lineart 151 MB", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_diffusers_xl_lineart.safetensors", "t2i-adapter_diffusers_xl_lineart.safetensors", "HF_TOKEN"),
]



In [112]:
run_download_list(controlnet_models_to_download, CONTROLNET_MODELS_DIR, "ControlNet")

#Checkpoints download




=== ControlNet ===
Parallel downloads: 3
[DOWNLOADING] t2i-adapter_xl_openpose 151 MB
[DOWNLOADING] t2i-adapter_xl_canny 148 MB
[DOWNLOADING] t2i-adapter_xl_sketch 148 MB
[OK]   t2i-adapter_xl_openpose 151 MB -> 150.7 MB
[DOWNLOADING] t2i-adapter_diffusers_xl_depth_midas 151 MB
[OK]   t2i-adapter_xl_canny 148 MB -> 147.9 MB
[DOWNLOADING] t2i-adapter_diffusers_xl_depth_zoe 151 MB
[OK]   t2i-adapter_xl_sketch 148 MB -> 147.9 MB
[DOWNLOADING] t2i-adapter_diffusers_xl_lineart 151 MB
[OK]   t2i-adapter_diffusers_xl_depth_zoe 151 MB -> 150.7 MB
[OK]   t2i-adapter_diffusers_xl_lineart 151 MB -> 150.7 MB
[OK]   t2i-adapter_diffusers_xl_depth_midas 151 MB -> 150.7 MB
Done: OK=6, FAIL=0


In [127]:
models_to_download = [
    ("WAI ILL V16.0 6,46 GB", "https://civitai.com/api/download/models/2514310?type=Model&format=SafeTensor&size=pruned&fp=fp16", "wai_v160.safetensors", "CIVITAI"),
]



In [128]:
run_download_list(models_to_download, MODELS_DIR, "Checkpoints")

#LoRa download




=== Checkpoints ===
Parallel downloads: 1
[DOWNLOADING] WAI ILL V16.0 6,46 GB
[FAIL] WAI ILL V16.0 6,46 GB -> 02/18 13:56:24 [ERROR] CUID#7 - Download aborted. URI=https://civitai.com/api/download/models/2514310?type=Model&format=SafeTensor&size=pruned&fp=fp16&token=739f0749f1aefc5f826f882be60b653c
Exception: [AbstractCommand.cc:351] errorCode=22 URI=https://b2.civitai.com/file/civitai-modelfiles/model/31176/waiNsfwIllustrious16.BHnI.safetensors?Authorization=3_20260218135624_ac13fff57a80b4bc2ff9036b_6bdfda04e25011bb998eaa95b6147d1bb89737d9_004_20260218145624_0049_dnld
  -> [HttpSkipResponseCommand.cc:239] errorCode=22 The response status is not successful. status=403

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
d3efa0|ERR |       0B/s|/workspace/stable-diffusion-webui-forge/models/Stable-diffusion/wai_v160.safetensors.part

Status Legend:
(ERR):error occurred.

aria2 will resume download if the tra

RuntimeError: Some downloads failed in Checkpoints: 1 item(s)

In [115]:
lora_to_download = [
    ("Detailer IL V2 218 MB",        "https://civitai.com/api/download/models/1736373?type=Model&format=SafeTensor",    "detailer_v2_il.safetensors",     "CIVITAI"),
    ("Realistic filter V1 55 MB",    "https://civitai.com/api/download/models/1124771?type=Model&format=SafeTensor",    "realistic_filter_v1_il.safetensors", "CIVITAI"),
    ("Hyperrealistic V4 ILL 435 MB", "https://civitai.com/api/download/models/1914557?type=Model&format=SafeTensor",    "hyperrealistic_v4_ill.safetensors",  "CIVITAI"),
    ("Niji semi realism V3.5 ILL 435 MB", "https://civitai.com/api/download/models/1882710?type=Model&format=SafeTensor", "niji_v35.safetensors", "CIVITAI"),
    ("ATNR Style ILL V1.1 350 MB", "https://civitai.com/api/download/models/1711464?type=Model&format=SafeTensor", "atnr_style_ill_v1.1.safetensors", "CIVITAI"),
    ("Face Enhancer Ill 218 MB", "https://civitai.com/api/download/models/1839268?type=Model&format=SafeTensor", "face_enhancer_ill.safetensors", "CIVITAI"),
    ("Smooth Detailer Booster V4 243 MB", "https://civitai.com/api/download/models/2196453?type=Model&format=SafeTensor", "smooth_detailer_booster_v4.safetensors", "CIVITAI"),
    ("USNR Style V-pred 157 MB", "https://civitai.com/api/download/models/2555444?type=Model&format=SafeTensor", "usnr_style.safetensors", "CIVITAI"),
    ("748cm Style V1 243 MB", "https://civitai.com/api/download/models/1056404?type=Model&format=SafeTensor", "748cm_style_v1.safetensors", "CIVITAI"),
    ("Velvet's Mythic Fantasy Styles IL 218 MB", "https://civitai.com/api/download/models/2620790?type=Model&format=SafeTensor", "velvets_styles.safetensors", "CIVITAI"),
    ("Pixel Art Style IL V7 435 MB", "https://civitai.com/api/download/models/2661972?type=Model&format=SafeTensor", "pixel_art.safetensors", "CIVITAI"),
]



In [116]:
run_download_list(lora_to_download, LORA_DIR, "LoRA")


=== LoRA ===
Parallel downloads: 3
[DOWNLOADING] Detailer IL V2 218 MB
[DOWNLOADING] Realistic filter V1 55 MB
[DOWNLOADING] Hyperrealistic V4 ILL 435 MB
[OK]   Detailer IL V2 218 MB -> 217.9 MB
[DOWNLOADING] Niji semi realism V3.5 ILL 435 MB
[OK]   Realistic filter V1 55 MB -> 54.8 MB
[DOWNLOADING] ATNR Style ILL V1.1 350 MB
[OK]   Niji semi realism V3.5 ILL 435 MB -> 435.4 MB
[DOWNLOADING] Face Enhancer Ill 218 MB
[OK]   ATNR Style ILL V1.1 350 MB -> 350.2 MB
[DOWNLOADING] Smooth Detailer Booster V4 243 MB
[OK]   Hyperrealistic V4 ILL 435 MB -> 435.4 MB
[DOWNLOADING] USNR Style V-pred 157 MB
[OK]   Face Enhancer Ill 218 MB -> 217.9 MB
[DOWNLOADING] 748cm Style V1 243 MB
[OK]   USNR Style V-pred 157 MB -> 156.9 MB
[DOWNLOADING] Velvet's Mythic Fantasy Styles IL 218 MB
[OK]   Smooth Detailer Booster V4 243 MB -> 243.2 MB
[DOWNLOADING] Pixel Art Style IL V7 435 MB
[OK]   748cm Style V1 243 MB -> 243.2 MB
[OK]   Pixel Art Style IL V7 435 MB -> 217.9 MB
[OK]   Velvet's Mythic Fantasy Sty

In [289]:
### RUN WEBUI (PURE PYTHON, BACKGROUND MODE) ###

import os


In [290]:
import re


In [291]:
import time


In [292]:
import subprocess


In [293]:
from pathlib import Path



In [294]:
forge_dir = Path(FORGE_DIR)


In [295]:
if not forge_dir.exists():
    raise FileNotFoundError(f"FORGE_DIR не найден: {forge_dir}")



In [296]:
launch_utils_path = forge_dir / "modules" / "launch_utils.py"


In [297]:
if launch_utils_path.exists():
    content = launch_utils_path.read_text(encoding="utf-8")

    clip_line_pattern = re.compile(
        r'^(?P<indent>\s*)run_pip\(f"install(?: --no-build-isolation)?(?: --no-use-pep517)? \{clip_package\}", "clip"\)\s*$',
        re.MULTILINE,
    )

    def _replace_clip_install(match):
        indent = match.group("indent")
        return (
            f'{indent}run_pip("install setuptools==69.5.1 wheel", "clip build deps")\n'
            f'{indent}run_pip(f"install --no-build-isolation {{clip_package}}", "clip")'
        )

    content_new, replacements = clip_line_pattern.subn(_replace_clip_install, content, count=1)

    numpy_fix_pattern = re.compile(
        r'^(?P<indent>\s*)startup_timer\.record\("install torch"\)\s*$',
        re.MULTILINE,
    )

    def _replace_torch_record(match):
        indent = match.group("indent")
        return (
            f"{indent}run_pip('install \"numpy==2.2.6\" \"scikit-image>=0.24.0\" --force-reinstall', 'numpy/skimage compatibility')\n"
            f'{indent}startup_timer.record("install torch")'
        )

    torch_fix_replacements = 0
    if "numpy/skimage compatibility" not in content_new:
        content_new, torch_fix_replacements = numpy_fix_pattern.subn(_replace_torch_record, content_new, count=1)
        if torch_fix_replacements:
            print("Patched torch stage (reinstalled numpy 2.x + scikit-image for ABI compatibility).")
        else:
            print("Torch-stage compatibility patch not applied (marker not found).")

    late_numpy_pattern = re.compile(r'^(?P<indent>\s*)import webui(?:\s*#.*)?\s*$', re.MULTILINE)

    def _replace_import_webui(match):
        indent = match.group("indent")
        return (
            f"{indent}run_pip('install \"numpy==2.2.6\" \"scikit-image>=0.24.0\" --force-reinstall', 'numpy/skimage late compatibility')\n"
            f'{indent}import webui'
        )

    late_fix_replacements = 0
    if "numpy/skimage late compatibility" not in content_new:
        content_new, late_fix_replacements = late_numpy_pattern.subn(_replace_import_webui, content_new, count=1)
        if late_fix_replacements:
            print("Patched start stage (re-pin numpy/scikit-image before importing webui).")
        else:
            print("Start-stage compatibility patch not applied (import webui line not found).")

    if replacements or torch_fix_replacements or late_fix_replacements:
        launch_utils_path.write_text(content_new, encoding="utf-8")

    if replacements:
        print("Patched CLIP install command (fixed flags + preserved indentation).")
    else:
        print("CLIP install patch already applied or target line not found.")
else:
    print(f"Warning: {launch_utils_path} not found, skipping CLIP patch.")

# Workaround for frequent startup issue in ControlNet: soft_inpainting.py


Patched CLIP install command (fixed flags + preserved indentation).


In [298]:
soft_inpainting_path = forge_dir / "extensions" / "sd-webui-controlnet" / "scripts" / "soft_inpainting.py"


In [299]:
soft_inpainting_disabled_path = soft_inpainting_path.with_suffix(".py.disabled")


In [300]:
if soft_inpainting_path.exists():
    os.replace(soft_inpainting_path, soft_inpainting_disabled_path)
    print(f"Disabled problematic script: {soft_inpainting_path.name} -> {soft_inpainting_disabled_path.name}")
else:
    print("soft_inpainting.py already disabled or not present.")



soft_inpainting.py already disabled or not present.


In [301]:
cmd = ["bash", "webui.sh", "-f", "--xformers", "--api", "--port", "17860"]


In [302]:
run_env = os.environ.copy()


In [303]:
run_env["MPLBACKEND"] = "Agg"


In [304]:
print("Running (background):", " ".join(cmd), "in", forge_dir)


Running (background): bash webui.sh -f --xformers --api --port 17860 in /workspace/stable-diffusion-webui-forge


In [305]:
print("MPLBACKEND forced to:", run_env["MPLBACKEND"])



MPLBACKEND forced to: Agg


In [306]:
venv_python = forge_dir / "venv" / "bin" / "python"


In [307]:
if venv_python.exists():
    print("Installing missing dependency in Forge venv: joblib")
    subprocess.run([str(venv_python), "-m", "pip", "install", "joblib"], check=False)

    print("Trying optional dependency in Forge venv: insightface")
    insightface_result = subprocess.run([str(venv_python), "-m", "pip", "install", "insightface"], check=False)
    if insightface_result.returncode != 0:
        print("Warning: insightface installation failed (optional dependency).")
else:
    print(f"Warning: venv python not found at {venv_python}, skipping joblib/insightface install.")



Installing missing dependency in Forge venv: joblib
Trying optional dependency in Forge venv: insightface


In [308]:
pid_path = forge_dir / "webui.pid"


In [309]:
log_path = forge_dir / "webui.log"




In [310]:
def _pid_alive(pid: int) -> bool:
    try:
        os.kill(pid, 0)
        return True
    except Exception:
        return False




In [311]:
def _log_tail(chars: int = 4000) -> str:
    if not log_path.exists():
        return "(webui.log not found yet)"
    return log_path.read_text(encoding="utf-8", errors="ignore")[-chars:]




In [312]:
running_pid = None


In [313]:
if pid_path.exists():
    try:
        old_pid = int(pid_path.read_text(encoding="utf-8").strip())
        if _pid_alive(old_pid):
            running_pid = old_pid
            print(f"WebUI already running with PID={old_pid}. Можно запускать последнюю ячейку.")
        else:
            pid_path.unlink(missing_ok=True)
    except Exception:
        pid_path.unlink(missing_ok=True)



In [314]:
started_proc = None


In [315]:
if running_pid is None:
    with open(log_path, "a", encoding="utf-8") as log_f:
        started_proc = subprocess.Popen(
            cmd,
            cwd=forge_dir,
            env=run_env,
            stdout=log_f,
            stderr=subprocess.STDOUT,
            start_new_session=True,
            text=True,
        )
    pid_path.write_text(str(started_proc.pid), encoding="utf-8")
    running_pid = started_proc.pid
    print(f"WebUI started in background. PID={running_pid}")
    print(f"Logs: {log_path}")



WebUI started in background. PID=8267
Logs: /workspace/stable-diffusion-webui-forge/webui.log


In [316]:
observe_timeout = int(os.environ.get("FORGE_STARTUP_OBSERVE_TIMEOUT", "120"))


In [317]:
observe_interval = float(os.environ.get("FORGE_STARTUP_OBSERVE_INTERVAL", "2"))



In [318]:
for _ in range(max(1, int(observe_timeout / observe_interval))):
    tail = _log_tail()

    if started_proc is not None:
        rc = started_proc.poll()
        if rc is not None:
            raise RuntimeError(
                f"WebUI process exited early with code {rc}.\n"
                f"Check logs at: {log_path}\n"
                f"Last log lines:\n{tail}"
            )

    if "Running on local URL" in tail or "Uvicorn running" in tail:
        print("WebUI appears ready. Можно запускать последнюю ячейку.")
        break

    time.sleep(observe_interval)
else:
    print("WebUI всё ещё запускается в фоне. Запускайте последнюю ячейку: она дождётся API.")
    print(f"Если API не поднимется, проверьте лог: {log_path}")



RuntimeError: WebUI process exited early with code 0.
Check logs at: /workspace/stable-diffusion-webui-forge/webui.log
Last log lines:
[0m     self.run_command(cmd_name)
  [31m   [0m   File "/tmp/pip-build-env-vj7lc9_t/overlay/lib/python3.12/site-packages/setuptools/_distutils/cmd.py", line 341, in run_command
  [31m   [0m     self.distribution.run_command(command)
  [31m   [0m   File "/tmp/pip-build-env-vj7lc9_t/overlay/lib/python3.12/site-packages/setuptools/dist.py", line 1107, in run_command
  [31m   [0m     super().run_command(command)
  [31m   [0m   File "/tmp/pip-build-env-vj7lc9_t/overlay/lib/python3.12/site-packages/setuptools/_distutils/dist.py", line 1019, in run_command
  [31m   [0m     cmd_obj.run()
  [31m   [0m   File "/tmp/pip-build-env-vj7lc9_t/overlay/lib/python3.12/site-packages/setuptools/command/build_ext.py", line 97, in run
  [31m   [0m     _build_ext.run(self)
  [31m   [0m   File "/tmp/pip-build-env-vj7lc9_t/overlay/lib/python3.12/site-packages

In [319]:
### API GENERATION FROM XLSX ###

import os


In [320]:
import re


In [321]:
import json


In [322]:
import glob


In [323]:
import time


In [324]:
import shutil


In [325]:
import zipfile


In [326]:
import base64


In [327]:
from datetime import datetime



In [328]:
import requests


In [329]:
from openpyxl import load_workbook



In [330]:
API_BASE = os.environ.get("FORGE_API_BASE", "http://127.0.0.1:17860")


In [331]:
TXT2IMG_URL = f"{API_BASE}/sdapi/v1/txt2img"


In [332]:
OPENAPI_URL = f"{API_BASE}/openapi.json"



In [333]:
PROMPTS_CANDIDATES = [
    os.path.join(GEN_DIR, "prompts.xlsx"),
    os.path.join(GEN_DIR, "prompts.xlxs"),
    os.path.join(GEN_DIR, "Prompts.xlsx"),
    os.path.join(GEN_DIR, "Prompts.xlxs"),
    os.path.join(BASE_DIR, "prompts.xlsx"),
    os.path.join(BASE_DIR, "prompts.xlxs"),
    os.path.join(BASE_DIR, "Prompts.xlsx"),
    os.path.join(BASE_DIR, "Prompts.xlxs"),
    "/workspace/gen/prompts.xlsx",
    "/workspace/gen/prompts.xlxs",
]



In [334]:
PROMPTS_CANDIDATES = list(dict.fromkeys(PROMPTS_CANDIDATES))



In [335]:
API_READY_TIMEOUT = int(os.environ.get("FORGE_API_READY_TIMEOUT", "600"))


In [336]:
API_READY_INTERVAL = float(os.environ.get("FORGE_API_READY_INTERVAL", "3"))




In [337]:
def wait_for_api_ready(base_url: str, timeout_s: int = API_READY_TIMEOUT, interval_s: float = API_READY_INTERVAL):
    start = time.time()
    last_error = None
    webui_log = os.path.join(FORGE_DIR, "webui.log")
    webui_pid = os.path.join(FORGE_DIR, "webui.pid")

    def _log_tail(chars: int = 4000):
        if not os.path.exists(webui_log):
            return "(webui.log not found yet)"
        with open(webui_log, "r", encoding="utf-8", errors="ignore") as f:
            return f.read()[-chars:]

    def _is_pid_alive():
        try:
            if not os.path.exists(webui_pid):
                return None
            with open(webui_pid, "r", encoding="utf-8") as f:
                pid = int(f.read().strip())
            os.kill(pid, 0)
            return pid
        except Exception:
            return False

    while (time.time() - start) < timeout_s:
        try:
            r = requests.get(f"{base_url}/sdapi/v1/progress", timeout=10)
            if r.status_code < 500:
                return True
            last_error = f"HTTP {r.status_code}"
        except Exception as e:
            last_error = str(e)

        pid_state = _is_pid_alive()
        if pid_state is False:
            raise RuntimeError(
                "Forge API недоступен и процесс WebUI уже завершился.\n"
                f"Last error: {last_error}\n"
                f"WebUI log tail:\n{_log_tail()}"
            )

        time.sleep(interval_s)

    raise RuntimeError(
        f"Forge API is not reachable at {base_url} after {timeout_s}s. "
        f"Last error: {last_error}.\n"
        f"WebUI log tail:\n{_log_tail()}\n"
        "Run the Forge startup cell first. If this is first launch, increase FORGE_API_READY_TIMEOUT (e.g. 900)."
    )




In [338]:
LOG_PATH = os.path.join(VOLUME_DIR, "log.txt")


In [339]:
API_IMAGES_DIR = os.path.join(OUTPUTS_DIR, "api_generated")


In [340]:
REQUEST_DUMPS_DIR = os.path.join(VOLUME_DIR, "request_dumps")


In [341]:
os.makedirs(API_IMAGES_DIR, exist_ok=True)


In [342]:
os.makedirs(REQUEST_DUMPS_DIR, exist_ok=True)



In [343]:
INPUT_IMAGE_DIRS = [
    os.path.join(GEN_DIR, "Images"),
    os.path.join(GEN_DIR, "images"),
]


In [344]:
INPUT_IMAGE_DIRS = [p for i, p in enumerate(INPUT_IMAGE_DIRS) if p and p not in INPUT_IMAGE_DIRS[:i]]



In [345]:
CONFLICT_RULES = [
    (("hr_resize_x", "hr_resize_y"), ("hr_scale",)),
]



In [346]:
CONTROLNET_FIELDS = [
    "enabled", "image", "mask", "weight", "module", "model", "resize_mode", "lowvram",
    "processor_res", "threshold_a", "threshold_b", "guidance_start", "guidance_end",
    "control_mode", "pixel_perfect",
]




In [347]:
def log_line(text: str):
    ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    line = f"[{ts}] {text}"
    print(line)
    with open(LOG_PATH, "a", encoding="utf-8") as f:
        f.write(line + "\n")




In [348]:
def normalize_value(value):
    if isinstance(value, str):
        stripped = value.strip()
        lowered = stripped.lower()
        if lowered in {"", "null", "none", "nan"}:
            return None
        if lowered in {"true", "yes", "1"}:
            return True
        if lowered in {"false", "no", "0"}:
            return False
        if re.fullmatch(r"-?\d+", stripped):
            return int(stripped)
        if re.fullmatch(r"-?\d+\.\d+", stripped):
            return float(stripped)
        return stripped
    return value




In [349]:
def first_existing_path(candidates):
    for p in candidates:
        if os.path.exists(p):
            return p
    return None




In [350]:
def parse_workbook(path):
    wb = load_workbook(path, data_only=True)
    ws = wb.active

    row1 = [normalize_value(v) for v in next(ws.iter_rows(min_row=1, max_row=1, values_only=True))]
    row2 = [normalize_value(v) for v in next(ws.iter_rows(min_row=2, max_row=2, values_only=True))]

    row1_headers_like = any(isinstance(v, str) and v in {"prompt", "negative_prompt", "sampler_name", "steps", "cn1_enabled"} for v in row1)

    if row1_headers_like:
        instruction = "{prompt}"
        headers = [str(v).strip() if v is not None else "" for v in row1]
        data_start_row = 2
    else:
        instruction_parts = [str(v).strip() for v in row1 if v is not None and str(v).strip()]
        if not instruction_parts:
            raise ValueError("Первая строка (инструкция) пустая")
        instruction = " ".join(instruction_parts)
        headers = [str(v).strip() if v is not None else "" for v in row2]
        data_start_row = 3

    if not any(headers):
        raise ValueError("Не найдены заголовки таблицы с переменными")

    rows = []
    for row_idx in range(data_start_row, ws.max_row + 1):
        row_values = [normalize_value(v) for v in next(ws.iter_rows(min_row=row_idx, max_row=row_idx, values_only=True))]
        if all(v is None for v in row_values):
            continue

        row_map = {}
        for idx, key in enumerate(headers):
            if not key or key.startswith("S_"):
                continue
            value = row_values[idx] if idx < len(row_values) else None
            if value is None:
                continue
            row_map[key] = value
        rows.append(row_map)

    if not rows:
        raise ValueError("Не найдено строк данных")

    return instruction, rows




In [351]:
def render_instruction(template: str, variables: dict):
    def repl(match):
        key = match.group(1)
        return str(variables.get(key, match.group(0)))
    return re.sub(r"\{([a-zA-Z0-9_]+)\}", repl, template)




In [352]:
def apply_conflict_rules(payload: dict):
    cleaned = dict(payload)
    for primary_keys, conflicting_keys in CONFLICT_RULES:
        primary_present = all((k in cleaned and cleaned[k] is not None) for k in primary_keys)
        if primary_present:
            for ck in conflicting_keys:
                cleaned.pop(ck, None)
    return cleaned




In [353]:
def fetch_txt2img_params_dump():
    try:
        response = requests.get(OPENAPI_URL, timeout=30)
        response.raise_for_status()
        spec = response.json()
        schemas = spec.get("components", {}).get("schemas", {})
        candidates = [
            "StableDiffusionTxt2ImgProcessingApi",
            "Txt2ImgRequest",
            "StableDiffusionProcessingTxt2Img",
        ]
        for name in candidates:
            if name in schemas:
                props = schemas[name].get("properties", {})
                return json.dumps({k: v.get("type", "unknown") for k, v in props.items()}, ensure_ascii=False, indent=2)
        return json.dumps(spec.get("paths", {}).get("/sdapi/v1/txt2img", {}), ensure_ascii=False, indent=2)
    except Exception as e:
        return f"Не удалось получить список переменных: {e}"




In [354]:
def resolve_image_path(image_ref):
    if image_ref is None:
        return None
    if isinstance(image_ref, str) and image_ref.startswith("data:image"):
        return image_ref

    ref = str(image_ref).strip()
    if not ref:
        return None

    if os.path.isabs(ref) and os.path.exists(ref):
        return ref

    candidates = [ref]
    if not os.path.splitext(ref)[1]:
        candidates += [f"{ref}.png", f"{ref}.jpg", f"{ref}.jpeg", f"{ref}.webp"]

    for base_dir in INPUT_IMAGE_DIRS:
        if not os.path.isdir(base_dir):
            continue
        for candidate in candidates:
            full = os.path.join(base_dir, candidate)
            if os.path.exists(full):
                return full

    for pattern in [f"**/{ref}", f"**/{ref}.*"]:
        for base_dir in INPUT_IMAGE_DIRS:
            if not os.path.isdir(base_dir):
                continue
            matches = glob.glob(os.path.join(base_dir, pattern), recursive=True)
            if matches:
                return matches[0]

    return None




In [355]:
def image_to_base64(image_ref):
    if image_ref is None:
        return None
    if isinstance(image_ref, str) and image_ref.startswith("data:image"):
        return image_ref

    image_path = resolve_image_path(image_ref)
    if not image_path:
        raise FileNotFoundError(f"Image/Mask not found: {image_ref}. Search dirs: {INPUT_IMAGE_DIRS}")

    ext = os.path.splitext(image_path)[1].lower()
    mime = {
        ".png": "image/png",
        ".jpg": "image/jpeg",
        ".jpeg": "image/jpeg",
        ".webp": "image/webp",
        ".bmp": "image/bmp",
    }.get(ext, "application/octet-stream")

    with open(image_path, "rb") as f:
        b64 = base64.b64encode(f.read()).decode("utf-8")
    return f"data:{mime};base64,{b64}"




In [356]:
def parse_override_settings(value):
    if value in (None, "", {}):
        return {}
    if isinstance(value, dict):
        return value
    if isinstance(value, str):
        txt = value.strip()
        if not txt:
            return {}
        if txt.startswith("{"):
            return json.loads(txt)
        return json.loads("{" + txt + "}")
    raise ValueError(f"Unsupported override_settings format: {type(value)}")



In [357]:
def deep_clean(value):
    if isinstance(value, dict):
        cleaned = {}
        for k, v in value.items():
            cv = deep_clean(v)
            if cv is None:
                continue
            if isinstance(cv, str) and not cv.strip():
                continue
            if isinstance(cv, dict) and not cv:
                continue
            cleaned[k] = cv
        return cleaned

    if isinstance(value, list):
        cleaned_list = []
        for item in value:
            ci = deep_clean(item)
            if ci is None:
                continue
            if isinstance(ci, str) and not ci.strip():
                continue
            if isinstance(ci, dict) and not ci:
                continue
            cleaned_list.append(ci)
        return cleaned_list

    if isinstance(value, str):
        stripped = value.strip()
        if stripped == "[]":
            return []
        if stripped.lower() in {"", "null", "none", "nan"}:
            return None
        return stripped

    return value




In [358]:
def dump_payload(payload, generation_idx):
    dump_path = os.path.join(REQUEST_DUMPS_DIR, f"request_{generation_idx:04d}.json")
    with open(dump_path, "w", encoding="utf-8") as f:
        json.dump(payload, f, ensure_ascii=False, indent=2)
    return dump_path




In [359]:
def build_payload(row_values, instruction_template):
    payload = dict(row_values)

    if "prompt" not in payload:
        payload["prompt"] = render_instruction(instruction_template, row_values)

    payload = {k: v for k, v in payload.items() if v is not None}

    if "override_settings" in payload:
        payload["override_settings"] = parse_override_settings(payload.get("override_settings"))

    controlnet_args = []
    for idx in (1, 2, 3):
        unit = {}
        for field in CONTROLNET_FIELDS:
            key = f"cn{idx}_{field}"
            if key in payload:
                unit[field] = payload.pop(key)

        if not unit:
            continue

        if "enabled" not in unit:
            unit["enabled"] = True

        if unit.get("image") is not None:
            unit["image"] = image_to_base64(unit.get("image"))
        if unit.get("mask") is not None:
            unit["mask"] = image_to_base64(unit.get("mask"))

        unit = deep_clean(unit)
        if unit.get("enabled"):
            controlnet_args.append(unit)

    if controlnet_args:
        payload.setdefault("alwayson_scripts", {})
        payload["alwayson_scripts"]["ControlNet"] = {"args": controlnet_args}

    payload = deep_clean(payload)
    payload = apply_conflict_rules(payload)
    payload = deep_clean(payload)
    return payload




In [360]:
def expected_image_count(payload):
    batch_size = int(payload.get("batch_size", 1) or 1)
    n_iter = int(payload.get("n_iter", 1) or 1)
    return max(1, batch_size) * max(1, n_iter)




In [361]:
def save_images_from_response(images_b64, generation_idx):
    saved = 0
    for i, b64 in enumerate(images_b64, start=1):
        image_data = b64.split(",", 1)[-1]
        file_name = f"gen_{generation_idx:04d}_{i:02d}.png"
        file_path = os.path.join(API_IMAGES_DIR, file_name)
        with open(file_path, "wb") as f:
            f.write(base64.b64decode(image_data))
        saved += 1
    return saved




In [362]:
def archive_outputs(tag: str):
    stamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    archive_name = os.path.join(VOLUME_DIR, f"outputs_{tag}_{stamp}.zip")
    with zipfile.ZipFile(archive_name, "w", compression=zipfile.ZIP_DEFLATED) as zf:
        for root, _, files in os.walk(OUTPUTS_DIR):
            for file in files:
                full_path = os.path.join(root, file)
                rel_path = os.path.relpath(full_path, OUTPUTS_DIR)
                zf.write(full_path, rel_path)

    for entry in os.listdir(OUTPUTS_DIR):
        p = os.path.join(OUTPUTS_DIR, entry)
        if os.path.isdir(p):
            shutil.rmtree(p)
        else:
            os.remove(p)
    os.makedirs(API_IMAGES_DIR, exist_ok=True)
    log_line(f"ARCHIVE created: {archive_name}. OUTPUTS_DIR cleaned.")




In [363]:
wait_for_api_ready(API_BASE)



RuntimeError: Forge API недоступен и процесс WebUI уже завершился.
Last error: HTTPConnectionPool(host='127.0.0.1', port=17860): Max retries exceeded with url: /sdapi/v1/progress (Caused by NewConnectionError("HTTPConnection(host='127.0.0.1', port=17860): Failed to establish a new connection: [Errno 111] Connection refused"))
WebUI log tail:
[0m     self.run_command(cmd_name)
  [31m   [0m   File "/tmp/pip-build-env-vj7lc9_t/overlay/lib/python3.12/site-packages/setuptools/_distutils/cmd.py", line 341, in run_command
  [31m   [0m     self.distribution.run_command(command)
  [31m   [0m   File "/tmp/pip-build-env-vj7lc9_t/overlay/lib/python3.12/site-packages/setuptools/dist.py", line 1107, in run_command
  [31m   [0m     super().run_command(command)
  [31m   [0m   File "/tmp/pip-build-env-vj7lc9_t/overlay/lib/python3.12/site-packages/setuptools/_distutils/dist.py", line 1019, in run_command
  [31m   [0m     cmd_obj.run()
  [31m   [0m   File "/tmp/pip-build-env-vj7lc9_t/overl

In [364]:
prompts_path = first_existing_path(PROMPTS_CANDIDATES)


In [365]:
if not prompts_path:
    raise FileNotFoundError(
        "Файл prompts.xlsx/prompts.xlxs не найден. Ожидались пути: " + ", ".join(PROMPTS_CANDIDATES)
    )



In [366]:
instruction_template, generation_rows = parse_workbook(prompts_path)


In [367]:
log_line(f"Loaded prompts file: {prompts_path}. Rows for generation: {len(generation_rows)}")


[2026-02-18 14:25:49] Loaded prompts file: /workspace/gen/Prompts.xlsx. Rows for generation: 14


In [368]:
log_line(f"Image search dirs: {INPUT_IMAGE_DIRS}")



[2026-02-18 14:25:50] Image search dirs: ['/workspace/gen/Images', '/workspace/gen/images']


In [369]:
syntax_error_dumped = False


In [370]:
images_since_archive = 0



In [371]:
total_expected_images = 0


In [372]:
total_saved_images = 0



In [373]:
for generation_idx, row_values in enumerate(generation_rows, start=1):
    try:
        payload = build_payload(row_values, instruction_template)
    except Exception as e:
        log_line(f"#{generation_idx} FAIL: payload build error: {e}")
        continue

    planned_images = expected_image_count(payload)
    total_expected_images += planned_images

    payload_dump_path = dump_payload(payload, generation_idx)
    log_line(f"#{generation_idx} request dump: {payload_dump_path}")
    if "alwayson_scripts" not in payload or not payload.get("alwayson_scripts", {}).get("ControlNet", {}).get("args"):
        log_line(f"#{generation_idx} INFO: no ControlNet image/mask attached in payload")

    try:
        response = requests.post(TXT2IMG_URL, json=payload, timeout=1800)
        if response.status_code >= 400:
            err_text = response.text[:1200]
            if not syntax_error_dumped:
                params_dump = fetch_txt2img_params_dump()
                log_line(f"#{generation_idx} FAIL syntax/validation: {response.status_code} {err_text}")
                log_line(f"#{generation_idx} FAIL payload: {payload_dump_path}")
                log_line("AVAILABLE PARAMS DUMP START")
                with open(LOG_PATH, "a", encoding="utf-8") as f:
                    f.write(params_dump + "\n")
                log_line("AVAILABLE PARAMS DUMP END")
                syntax_error_dumped = True
            else:
                log_line(f"#{generation_idx} FAIL: {response.status_code} {err_text}")
                log_line(f"#{generation_idx} FAIL payload: {payload_dump_path}")
            continue

        result = response.json()
        images = result.get("images", []) or []
        saved_now = save_images_from_response(images, generation_idx)
        total_saved_images += saved_now
        images_since_archive += saved_now

        log_line(f"#{generation_idx} OK images={saved_now} expected={planned_images}")

        if images_since_archive >= 15:
            archive_outputs(tag=f"part_{generation_idx:04d}")
            images_since_archive = 0

    except requests.exceptions.Timeout:
        log_line(f"#{generation_idx} FAIL: timeout (possible heavy generation or API freeze)")
        log_line(f"#{generation_idx} FAIL payload: {payload_dump_path}")
    except requests.exceptions.RequestException as e:
        reason = str(e)
        if "out of memory" in reason.lower() or "oom" in reason.lower():
            reason = "OOM"
        log_line(f"#{generation_idx} FAIL: {reason}")
        log_line(f"#{generation_idx} FAIL payload: {payload_dump_path}")
    except Exception as e:
        log_line(f"#{generation_idx} FAIL: unexpected error: {e}")
        log_line(f"#{generation_idx} FAIL payload: {payload_dump_path}")



[2026-02-18 14:25:57] #1 request dump: /workspace/volume/request_dumps/request_0001.json
[2026-02-18 14:25:57] #1 INFO: no ControlNet image/mask attached in payload
[2026-02-18 14:25:57] #1 FAIL: HTTPConnectionPool(host='127.0.0.1', port=17860): Max retries exceeded with url: /sdapi/v1/txt2img (Caused by NewConnectionError("HTTPConnection(host='127.0.0.1', port=17860): Failed to establish a new connection: [Errno 111] Connection refused"))
[2026-02-18 14:25:57] #1 FAIL payload: /workspace/volume/request_dumps/request_0001.json
[2026-02-18 14:25:57] #2 request dump: /workspace/volume/request_dumps/request_0002.json
[2026-02-18 14:25:57] #2 INFO: no ControlNet image/mask attached in payload
[2026-02-18 14:25:57] #2 FAIL: HTTPConnectionPool(host='127.0.0.1', port=17860): Max retries exceeded with url: /sdapi/v1/txt2img (Caused by NewConnectionError("HTTPConnection(host='127.0.0.1', port=17860): Failed to establish a new connection: [Errno 111] Connection refused"))
[2026-02-18 14:25:57] #

In [374]:
if images_since_archive > 0:
    archive_outputs(tag="final")



In [375]:
log_line(f"Generation cycle completed. Requests={len(generation_rows)} expected_images={total_expected_images} saved_images={total_saved_images}")



[2026-02-18 14:26:04] Generation cycle completed. Requests=14 expected_images=18 saved_images=0


In [376]:
import requests



In [377]:
url = "http://127.0.0.1:17860" # замените на ваш адрес

# Получить список текущих настроек (options)


In [378]:
options = requests.get(f'{url}/sdapi/v1/options').json()


ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=17860): Max retries exceeded with url: /sdapi/v1/options (Caused by NewConnectionError("HTTPConnection(host='127.0.0.1', port=17860): Failed to establish a new connection: [Errno 111] Connection refused"))

In [ ]:
print(options.keys()) # Выведет все доступные имена переменных для настроек

# Получить данные о текущей модели


In [ ]:
models = requests.get(f'{url}/sdapi/v1/sd-models').json()


In [ ]:
print(models)

In [ ]:
print("Найденные процессы Forge перед убийством:")
subprocess.run(["ps", "aux"], capture_output=False)  # покажет весь вывод

# Убиваем
subprocess.run(["pkill", "-f", "stable-diffusion-webui-forge"], check=False)
subprocess.run(["pkill", "-f", "launch.py"], check=False)
subprocess.run(["pkill", "-f", "webui.sh"], check=False)

print("\nПосле убийства:")
subprocess.run(["ps", "aux"], capture_output=False)

In [287]:
result = subprocess.run(
    ["ps", "aux"],
    capture_output=True,
    text=True
)

# Фильтруем только релевантные строки (чтобы не выводить весь ps aux)
lines = result.stdout.splitlines()
forge_processes = [line for line in lines if any(word in line for word in ["forge", "webui.sh", "launch.py", "stable-diffusion-webui-forge", "relauncher"]) and "grep" not in line]

if forge_processes:
    print("Forge запущен! Найдены процессы:")
    for proc in forge_processes:
        print(proc)
else:
    print("Forge НЕ запущен (или процессы не найдены по ключевым словам).")

Forge запущен! Найдены процессы:
root        7759  3.5  0.0   4708  3328 ?        S    14:19   0:00 /bin/bash /opt/supervisor-scripts/forge.sh
root        7761  0.0  0.0   3088  1536 ?        S    14:19   0:00 tee -a /var/log/portal/forge.log


In [288]:
import subprocess

# Пробуем ss (чаще всего работает в Ubuntu-контейнерах Vast.ai)
try:
    result = subprocess.run(["ss", "-tuln"], capture_output=True, text=True, timeout=5)
    if result.returncode == 0:
        print("Открытые TCP-порты (ss):\n")
        print(result.stdout)
    else:
        print("ss не сработал")
except:
    print("ss недоступен")

Открытые TCP-порты (ss):

Netid State  Recv-Q Send-Q Local Address:Port  Peer Address:PortProcess
tcp   LISTEN 0      100        127.0.0.1:35921      0.0.0.0:*          
tcp   LISTEN 0      100        127.0.0.1:33493      0.0.0.0:*          
tcp   LISTEN 0      100        127.0.0.1:55111      0.0.0.0:*          
tcp   LISTEN 0      100        127.0.0.1:54521      0.0.0.0:*          
tcp   LISTEN 0      100        127.0.0.1:38271      0.0.0.0:*          
tcp   LISTEN 0      100        127.0.0.1:60811      0.0.0.0:*          
tcp   LISTEN 0      128          0.0.0.0:22         0.0.0.0:*          
tcp   LISTEN 0      128          0.0.0.0:8080       0.0.0.0:*          
tcp   LISTEN 0      128             [::]:22            [::]:*          

